In [1]:
import os
import shutil
import cv2
import tensorflow as tf
import numpy as np
from tkinter import *
from tkinter.ttk import *
import tkinter as tk
import json
import collections
import time
CATEGORIES = ["A", "B", "C", "D", "E", "F", "G", "H", "I","K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "?"]

In [2]:
model = tf.keras.models.load_model("modelfinalpls.h5")

In [3]:
def makedir(directory):
    # If folder doesn't exist create it
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
         pass

In [4]:
def cr8data():

    cap = cv2.VideoCapture(0)
    SIZE_IMG = 200
    i=0
    image_count = 0
    gesture_num = 0
    gest_dir = ' '
    
    # Delete folder and its contents if it already exists.
    if os.path.exists('handgestures'):
        shutil.rmtree('handgestures')

    while True:
        while i < 4:

            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)

            #define region of interest
            roi = frame[100:400, 320:620]
            roi = cv2.resize(roi, (SIZE_IMG, SIZE_IMG), interpolation = cv2.INTER_AREA)
            copy = frame.copy()
            cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)

            if gesture_num > 23:
                break
            if i == 0:
                image_count = 0
                cv2.putText(copy, "Hit enter to record gesture " + str(CATEGORIES[gesture_num]), (100 , 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
                cv2.putText(copy, "Hit ESC to exit ", (40 , 65), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
                if cv2.waitKey(1) == 27: #ESC
                    break

            if i == 1:
                image_count+=1
                if image_count == 3000:
                    image_count = 0
                    i+=1                
                cv2.putText(copy, "Recording gesture "+ str(CATEGORIES[gesture_num]) + " - Train(3000)", (20 , 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
                cv2.putText(copy, str(image_count), (450 , 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
                gest_dir = ('./handgestures/train/{}/'.format(CATEGORIES[gesture_num]))
                makedir(gest_dir)
                cv2.imwrite(gest_dir + (CATEGORIES[gesture_num]+str(image_count+1)) + ".jpg", roi)

            if i == 2:
                image_count+=1
                if image_count == 1000:
                    image_count = 0
                    i+=1
                cv2.putText(copy, "Recording gesture "+ str(CATEGORIES[gesture_num]) + " - Test (1000)", (20 , 40), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
                cv2.putText(copy, str(image_count), (450 , 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
                gest_dir = ('./handgestures/test/{}/'.format(CATEGORIES[gesture_num]))
                makedir(gest_dir)
                cv2.imwrite(gest_dir + (CATEGORIES[gesture_num]+str(image_count+1)) + ".jpg", roi)
            if i == 3:
                gesture_num = gesture_num + 1
                i = 0
            cv2.imshow('Dataset', copy)
            if cv2.waitKey(1) == 13: #Enter
                image_count = 0
                i+=1
        break

    cap.release()
    cv2.destroyAllWindows() 

In [5]:
def predict(temp):
    IMG_SIZE = 50
    cap = cv2.VideoCapture(0)
    while True:
        
        ret, frame1 = cap.read()
        
        ##############################
        frame=cv2.flip(frame1, 1)
        
        #define region of interest
        roi = frame[100:400, 320:620]
        
        roi = cv2.resize(roi, (IMG_SIZE, IMG_SIZE))

        copy = frame.copy()
        cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)
        
        roi = roi.reshape(-1,IMG_SIZE,IMG_SIZE,3) 
        
        prediction1 = model.predict(roi)
        x1=int(prediction1.argmax(axis=1))
        if x1 > 24 or x1 < 0:
            x1 = 24        
        
        pls = (CATEGORIES[x1])
        
        cv2.putText(copy, pls, (440 , 90), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 0, 0), 2)
        
        if temp == 0:
            start_time = time.time()
            Text = ' '
            db = collections.deque('pls')
            db.append(pls)
            db2 = []
            temp = 1
        
        elapsed = int((time.time()-start_time))
        
        if db[0] != db[1]:
            start_time = time.time()

        if elapsed != 3:
            db.append(pls)
        if len(db)>3:
            db.popleft()
        if elapsed == 3:
            db2.append(db[1])
            db2_index = ((len(db2))-1)
            Text = str(Text + db2[db2_index])
            start_time = time.time()
        cv2.putText(copy, str(elapsed), (600 , 50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 255), 2)
        cv2.putText(copy, Text, (-30 , 50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 255, 255), 2) #Text
        
        if cv2.waitKey(1) == 9: #TAB
            Text = ' '
        if cv2.waitKey(1) == 27: #ESC
            break
            
        cv2.imshow('Prediction', copy)    
    
    cap.release()
    cv2.destroyAllWindows()

In [6]:
def main():
    # creates a Tk() object
    master = Tk()
    # sets the geometry of main
    # root window
    master.geometry("350x260")
    master.title('Sign language gesture recognition')

    def predict2():
        predict(0)
    
    # a button widget which will open a new window on button click
    btn1 = Button(master, text ="Open prediction window", command = (predict2))
    btn1.pack(pady = 10)

    
    btn2 = Button(master, text ="Make a new dataset", command = cr8data)
    btn2.pack(pady = 10)
        
    label1 = Label(master, text ="Graduation Project Dec. 2021")
    label1.pack(pady = 20)
    label2 = Label(master, text ="By: Shady Mahmoud Ahmed Elkholy")
    label2.pack(pady = 0)
    label3 = Label(master, text ="ID: 1751710378")
    label3.pack(pady = 0)
    label4 = Label(master, text ="Supervised by: Dr. Sara Nabil")
    label4.pack(pady = 20)
    
    # mainloop, runs infinitely
    mainloop()

main()